# MLOps with TensorFlow (using TensorFlow-Extended)

Training a machine learning model is only half of the journey. A model to be useful in the real-world you need to deploy it (i.e. productionize it). Productionization is the process of deploying the model on some infrastructure (e.g. a single machine or a cluster) and making it accessible for predicting via an web based API (e.g. REST).

<table align="left">
    <td>
        <a target="_blank" href="https://colab.research.google.com/github/thushv89/manning_tf2_in_action/blob/master/Ch14/14.1_MLOps_with_tensorflow"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
    </td>
</table>



In [89]:
import tensorflow as tf
#import tensorflow_hub as hub
import requests
print(tf.__version__)
import requests
import os
import time
import pandas as pd

assert_msg = "Pandas version should be < 1.0 for tfx"
assert pd.__version__.startswith('0.')

import random
import shutil
import tempfile
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import CSVLogger
import numpy as np
import pickle
from tensorflow.keras.models import load_model, Model
import glob
from functools import partial
import absl

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except:
        print("Couldn't set memory_growth")
        pass
    
    
def fix_random_seed(seed):
    """ Setting the random seed of various libraries """
    try:
        np.random.seed(seed)
    except NameError:
        print("Warning: Numpy is not imported. Setting the seed for Numpy failed.")
    try:
        tf.random.set_seed(seed)
    except NameError:
        print("Warning: TensorFlow is not imported. Setting the seed for TensorFlow failed.")
    try:
        random.seed(seed)
    except NameError:
        print("Warning: random module is not imported. Setting the seed for random failed.")

# Fixing the random seed
random_seed=4321
fix_random_seed(random_seed)

2.4.2


## Caveats

* Pandas >= 1.0 will give,
  * `ValueError: Value must be a nonnegative integer or None` when displaying results (e.g. context.show() calls)
* Had to install apache-beam 2.29 (not 2.30) coz 2.30 was giving the error
  * `Attribute error: type StringMethods does not have attribute fullmatch`

## Download the data

We are going to use a dataset contining information about forest fires in the USA. The dataset is downloaded from [this link](http://archive.ics.uci.edu/ml/datasets/Forest+Fires). The dataset contains the following features.

* X - x-axis spatial coordinate within the Montesinho park map
* Y - y-axis spatial coordinate within the Montesinho park map
* month - month of the year
* day - day of the week
* FFMC - Fine Fuel Moisture Code (FFMC) represents fuel moisture of forest litter fuels under the shade of a forest canopy
* DMC - DMC is a numerical rating of the average moisture content of soil
* DC - The Drought Code (DC) represents drying deep into the soil.
* ISI - ISI index from the FWI system (The Initial Spread Index (ISI) is an expected rate of fire spread)
* temp - temperature in Celsius degrees
* RH - relative humidity in %
* wind - wind speed in km/h
* rain - outside rain in mm/m2
* area - the burned area of the forest (in ha)

In [90]:
# Downloading the data

import os
import requests
import tarfile

import shutil

# Retrieve the data
if not os.path.exists(os.path.join('data', 'csv', 'forestfires.csv')):
    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.csv"
    # Get the file from web
    r = requests.get(url)

    if not os.path.exists(os.path.join('data', 'csv')):
        os.makedirs(os.path.join('data', 'csv'))
    
    # Write to a file
    with open(os.path.join('data', 'csv', 'forestfires.csv'), 'wb') as f:
        f.write(r.content)
        
else:
    print("The forestfires.csv file already exists.")
    
    
if not os.path.exists(os.path.join('data', 'forestfires.names')):
    
    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.names"
    # Get the file from web
    r = requests.get(url)

    if not os.path.exists('data'):
        os.makedirs('data')
    
    # Write to a file
    with open(os.path.join('data', 'forestfires.names'), 'wb') as f:
        f.write(r.content)
        
else:
    print("The forestfires.names file already exists.")



The forestfires.csv file already exists.
The forestfires.names file already exists.


## Separate train/valid and test datasets

In [91]:
import pandas as pd

df = pd.read_csv(os.path.join('data', 'csv', 'forestfires.csv'), index_col=None, header=0)
train_df = df.sample(frac=0.95, random_state=random_seed)
test_df = df.loc[~df.index.isin(train_df.index), :]

train_path = os.path.join('data','csv','train')
os.makedirs(train_path, exist_ok=True)
test_path = os.path.join('data','csv','test')
os.makedirs(test_path, exist_ok=True)

train_df.to_csv(os.path.join(train_path, 'forestfires.csv'), index=False, header=True)
test_df.to_csv(os.path.join(test_path, 'forestfires.csv'), index=False, header=True)

## Defining the pipeline root

In [92]:
# This is the directory containing the TFX Chicago Taxi Pipeline example.
_pipeline_root = os.path.join(os.getcwd(), 'pipeline', 'examples', 'forest_fires_pipeline')
print(_pipeline_root)

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline


## Defining an interactive context

Context runs various steps in the pipeline and maintains state between different steps. Interactive means that, it can be called/run via multiple calls, instead of doing via a single call. This is suited when running things on a notebook, where code is quite isolated.

In [93]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.

context = InteractiveContext(pipeline_name = "forest_fires", pipeline_root=_pipeline_root)

## Step 1: Define an Example Generator

This example generator generates examples (as `TFRecord` objects) from a CSV file. 

In [94]:
from tfx.components import CsvExampleGen
from tfx.proto import example_gen_pb2

# You can change the configuration of generating train-eval split by passing a SplitConfig as below
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=3),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
    ]))

# Define the example generator
example_gen = CsvExampleGen(input_base=os.path.join('data', 'csv', 'train'))
# Run the example generator
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Processing input csv data data/csv/train/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:26223,xor_checksum:1626299480,sum_checksum:1626299480"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Difference channel and a pb2 (that's a protobuf)

https://www.tensorflow.org/tfx/api_docs/python/tfx/types/Channel (explain channels)

## Some results produced by `CsvExampleGen`

In [95]:
# Print outputs
print(example_gen.outputs)

# Accessing various variables in the example generator
artifact = example_gen.outputs['examples'].get()[0]

# Print some of that
print("Artifact split names: {}".format(artifact.split_names))
print("Artifact URI: {}".format(artifact.uri))


{'examples': Channel(
    type_name: Examples
    artifacts: [Artifact(artifact: id: 1
type_id: 5
uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/CsvExampleGen/examples/1"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:26223,xor_checksum:1626299480,sum_checksum:1626299480"
  }
}
custom_properties {
  key: "payload_format"
  value {
    string_value: "FORMAT_TF_EXAMPLE"
  }
}
custom_properties {
  key: "span"
  value {
    int_value: 0
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "0.30.0"
  }
}
state: LIVE
, artifact_type: id: 5
name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties {
  key: "version"
 

## Print a few examples generated from the CSV file

In [96]:
import pprint

# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 2 records and decode them.
for tfrecord in dataset.take(2):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  print(example)


features {
  feature {
    key: "DC"
    value {
      float_list {
        value: 605.7999877929688
      }
    }
  }
  feature {
    key: "DMC"
    value {
      float_list {
        value: 141.3000030517578
      }
    }
  }
  feature {
    key: "FFMC"
    value {
      float_list {
        value: 95.0999984741211
      }
    }
  }
  feature {
    key: "ISI"
    value {
      float_list {
        value: 17.700000762939453
      }
    }
  }
  feature {
    key: "RH"
    value {
      int64_list {
        value: 43
      }
    }
  }
  feature {
    key: "X"
    value {
      int64_list {
        value: 5
      }
    }
  }
  feature {
    key: "Y"
    value {
      int64_list {
        value: 4
      }
    }
  }
  feature {
    key: "area"
    value {
      float_list {
        value: 2.0
      }
    }
  }
  feature {
    key: "day"
    value {
      bytes_list {
        value: "tue"
      }
    }
  }
  feature {
    key: "month"
    value {
      bytes_list {
        value: "aug"
    

## Step 2: Generate statistics of the data

Statistics compute various metrics. For, 
* continuous variables
  * like mean, standard deviation, count of zeros, median, min/max, etc.  
* categorical variables
  * count, missing count, number of unique values, etc.

In [97]:
from tfx.components import StatisticsGen

# Define the statistic generator
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
# Run the statistic  generator
context.run(statistics_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/StatisticsGen/statistics/2/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/StatisticsGen/statistics/2/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## View the statistics of train / eval data

In [98]:
context.show(statistics_gen.outputs['statistics'])

## Step 3: Infer the schema of the data

In [99]:
from tfx.components import SchemaGen

schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## Print `schema_gen` artifact location

In [100]:
print(schema_gen.outputs['schema']._artifacts[0].uri)

/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/SchemaGen/schema/3


## Display the inferred schema

TFX uses the `tensorflow_data_validation` library to do that. To do this manually, do the following.

```
schema = tfdv.load_schema_text(
    os.path.join(schema_gen.outputs['schema']._artifacts[0].uri, 'schema.pbtxt')
) 
features_df, domain_df = tfdv.utils.display_util.get_schema_dataframe(schema)
tfdv.utils.display_util.display(domain_df)
```

In [101]:
import tensorflow_data_validation as tfdv

context.show(schema_gen.outputs['schema'])


,Type,Presence,Valency,Domain
Feature name,,,,
'day',STRING,required,single,'day'
'month',STRING,required,single,'month'
'DC',FLOAT,required,single,-
'DMC',FLOAT,required,single,-
'FFMC',FLOAT,required,single,-
'ISI',FLOAT,required,single,-
'RH',INT,required,single,-
'X',INT,required,single,-
'Y',INT,required,single,-


,Values
Domain,
'day',"'fri', 'mon', 'sat', 'sun', 'thu', 'tue', 'wed'"
'month',"'apr', 'aug', 'dec', 'feb', 'jan', 'jul', 'jun', 'mar', 'may', 'oct', 'sep', 'nov'"


https://www.tensorflow.org/tfx/tutorials/tfx/components

## Step 4: Transforming data into features with `tensorflow_transform`

### Defining feature types

In [102]:
%%writefile forest_fires_constants.py

# Vocab based features

VOCAB_FEATURE_KEYS = ['day','month']

# Vocab based features are in fact categorical features
# Categorical features are assumed to each have a maximum value in the dataset
MAX_CATEGORICAL_FEATURE_VALUES = [7, 12]

# Dense features (these will go to the model as they are)

DENSE_FLOAT_FEATURE_KEYS = ['DC', 'DMC', 'FFMC', 'ISI', 'rain', 'temp', 'wind', 'X', 'Y']

# Bucketized features 

# Number of buckets used by tf.transform for encoding each feature.
BUCKET_FEATURE_BOUNDARIES = [(33, 66)]
BUCKET_FEATURE_KEYS = ['RH']

# Label features

LABEL_KEY = 'area'


def transformed_name(key):
    """ Define a function that will add a suffix to the feature name """
    
    return key + '_xf'


Overwriting forest_fires_constants.py


### Processing the columns to features

In [103]:
%%writefile forest_fires_transform.py

import tensorflow as tf
import tensorflow_transform as tft

import forest_fires_constants

_DENSE_FLOAT_FEATURE_KEYS = forest_fires_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = forest_fires_constants.VOCAB_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = forest_fires_constants.BUCKET_FEATURE_KEYS
_BUCKET_FEATURE_BOUNDARIES = forest_fires_constants.BUCKET_FEATURE_BOUNDARIES
_LABEL_KEY = forest_fires_constants.LABEL_KEY
_transformed_name = forest_fires_constants.transformed_name


def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.
  Args:
    inputs: map from feature keys to raw not-yet-transformed features.
  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
    
  # Treating dense features
  for key in _DENSE_FLOAT_FEATURE_KEYS:
    # Preserve this feature as a dense float, setting nan's to the mean.
    outputs[_transformed_name(key)] = tft.scale_to_z_score(
        sparse_to_dense(inputs[key])
    )

  # Treating categorical features  
  for key in _VOCAB_FEATURE_KEYS:
    # Build a vocabulary for this feature.
    outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(
        sparse_to_dense(inputs[key]),
        num_oov_buckets=1)

  # Treating bucketized features
  for key, boundary in zip(_BUCKET_FEATURE_KEYS, _BUCKET_FEATURE_BOUNDARIES):
    outputs[_transformed_name(key)] = tft.apply_buckets(
        sparse_to_dense(inputs[key]), bucket_boundaries=[boundary])

  # Treating the label feature
  outputs[_transformed_name(_LABEL_KEY)] = sparse_to_dense(inputs[_LABEL_KEY])

  return outputs

def sparse_to_dense(x):
    
    return tf.squeeze(
        tf.sparse.to_dense(tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1])),
        axis=1
    )


Overwriting forest_fires_transform.py


### Running the transform step

In [104]:
from tfx.components import Transform

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath('forest_fires_transform.py'),
)

context.run(transform)


INFO:absl:Generating ephemeral wheel package for '/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/forest_fires_transform.py' (including modules: ['forest_fires_constants', 'forest_fires_transform', 'forest_fires_trainer']).
INFO:absl:User module package has hash fingerprint version 626064dad212184db8b5fe12b96d6ebcf4204f9254387758f58ee8edf2b06004.
INFO:absl:Executing: ['/home/thushv89/anaconda3/envs/manning.tf2.tfx/bin/python', '/tmp/tmpfer1d6h8/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpyfwp7flb', '--dist-dir', '/tmp/tmppsho2q3g']
INFO:absl:Successfully built user code wheel distribution at '/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/_wheels/tfx_user_code_Transform-0.0+626064dad212184db8b5fe12b96d6ebcf4204f9254387758f58ee8edf2b06004-py3-none-any.whl'; target user module is 'forest_fires_transform'.
INFO:absl:Full user module path is 'forest_fires_transform@/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipelin

INFO:absl:Feature day has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature month has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature FFMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature ISI has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature RH has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature X has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Y has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature area has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature rain has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature temp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature wind has no shape. Setting to VarLenSparseTensor.


INFO:absl:Feature day has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature month has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature FFMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature ISI has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature RH has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature X has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Y has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature area has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature rain has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature temp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature wind has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature day has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature month has no shape. Setting to VarLenSparse

INFO:absl:Feature day has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature month has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature FFMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature ISI has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature RH has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature X has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Y has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature area has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature rain has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature temp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature wind has no shape. Setting to VarLenSparseTensor.


INFO:absl:Feature day has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature month has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature FFMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature ISI has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature RH has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature X has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Y has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature area has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature rain has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature temp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature wind has no shape. Setting to VarLenSparseTensor.


...

INFO:tensorflow:Assets written to: /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Transform/transform_graph/4/.temp_path/tftransform_tmp/e92ad7c2dee946cf959391dd7046cd03/assets


INFO:tensorflow:Assets written to: /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Transform/transform_graph/4/.temp_path/tftransform_tmp/d4ad9466f14742f182a61e6ea8884fcd/assets


INFO:tensorflow:Assets written to: /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Transform/transform_graph/4/.temp_path/tftransform_tmp/d4ad9466f14742f182a61e6ea8884fcd/assets


INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 4
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 4
        type_id: 11
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Transform/transform_graph/4"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 5
        type_id: 5
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Transform/transformed_examples/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 6
        type_id: 12
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Transform/updated_analyzer_cache/4"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 12
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Print a few examples generated from the transformer

In [105]:
import pprint

import forest_fires_constants

_DENSE_FLOAT_FEATURE_KEYS = forest_fires_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = forest_fires_constants.VOCAB_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = forest_fires_constants.BUCKET_FEATURE_KEYS
_LABEL_KEY = forest_fires_constants.LABEL_KEY
pp = pprint.PrettyPrinter()

# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.

example_records = []
float_features = [f+'_xf' for f in _DENSE_FLOAT_FEATURE_KEYS + ['area']]
int_features = [f+'_xf' for f in _BUCKET_FEATURE_KEYS + _VOCAB_FEATURE_KEYS]
for tfrecord in dataset.take(5):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  record = [example.features.feature[f].int64_list.value for f in int_features] + [example.features.feature[f].float_list.value for f in float_features]
  example_records.append(record)
  print(example)
  print("="*50)

features {
  feature {
    key: "DC_xf"
    value {
      float_list {
        value: 0.18325766921043396
      }
    }
  }
  feature {
    key: "DMC_xf"
    value {
      float_list {
        value: 0.4082949757575989
      }
    }
  }
  feature {
    key: "FFMC_xf"
    value {
      float_list {
        value: 0.7053141593933105
      }
    }
  }
  feature {
    key: "ISI_xf"
    value {
      float_list {
        value: 1.7617137432098389
      }
    }
  }
  feature {
    key: "RH_xf"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "X_xf"
    value {
      float_list {
        value: 0.16529320180416107
      }
    }
  }
  feature {
    key: "Y_xf"
    value {
      float_list {
        value: -0.23549708724021912
      }
    }
  }
  feature {
    key: "area_xf"
    value {
      float_list {
        value: 2.0
      }
    }
  }
  feature {
    key: "day_xf"
    value {
      int64_list {
        value: 5
      }
    }
  }
  feature {
    key: 

### Presenting as a dataframe

In [106]:
sample_df = pd.DataFrame.from_records(example_records, columns=int_features+float_features)
sample_df.head()

,RH_xf,day_xf,month_xf,DC_xf,DMC_xf,FFMC_xf,ISI_xf,rain_xf,temp_xf,wind_xf,X_xf,Y_xf,area_xf
0,[1],[5],[0],[0.18325766921043396],[0.4082949757575989],[0.7053141593933105],[1.7617137432098389],[-0.08982300013303757],[0.8269943594932556],[1.3554872274398804],[0.16529320180416107],[-0.23549708724021912],[2.0]
1,[1],[4],[0],[0.06929662823677063],[0.25802576541900635],[0.7210833430290222],[0.25552359223365784],[-0.08982300013303757],[0.22984977066516876],[-0.9850294589996338],[1.4553557634353638],[1.3173850774765015],[2.549999952316284]
2,[1],[1],[0],[0.6091344356536865],[1.7497596740722656],[0.6580066084861755],[0.5856475234031677],[-0.08982300013303757],[0.6864895224571228],[-0.47125762701034546],[1.0253349542617798],[-0.23549708724021912],[0.0]
3,[0],[6],[0],[0.5386473536491394],[1.6041864156723022],[0.7210833430290222],[1.823611855506897],[-0.08982300013303757],[2.003720283508301],[0.32794323563575745],[-1.1247694492340088],[-0.23549708724021912],[0.0]
4,[1],[0],[8],[-2.265638589859009],[-1.5749441385269165],[0.05876835808157921],[0.6475457549095154],[-0.08982300013303757],[-0.999565839767456],[3.125145673751831],[0.595314085483551],[-1.011938214302063],[61.130001068115234]


## Step 5: Defining the Trainer

In [107]:
%%writefile forest_fires_trainer.py

# Classif vs regression signature defs
# https://www.tensorflow.org/tfx/serving/signature_defs

from typing import List, Text

import os
import absl
import datetime
import tensorflow as tf
import tensorflow_transform as tft

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

import forest_fires_constants


_DENSE_FLOAT_FEATURE_KEYS = forest_fires_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = forest_fires_constants.VOCAB_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = forest_fires_constants.BUCKET_FEATURE_KEYS
_BUCKET_FEATURE_BOUNDARIES = forest_fires_constants.BUCKET_FEATURE_BOUNDARIES
_MAX_CATEGORICAL_FEATURE_VALUES = forest_fires_constants.MAX_CATEGORICAL_FEATURE_VALUES
_LABEL_KEY = forest_fires_constants.LABEL_KEY
_transformed_name = forest_fires_constants.transformed_name


def _transformed_names(keys):
  return [_transformed_name(key) for key in keys]


def _get_serve_tf_examples_fn(model, tf_transform_output):
  """Returns a function that parses a serialized tf.Example and applies TFT."""

  model.tft_layer = tf_transform_output.transform_features_layer()

  @tf.function
  def serve_tf_examples_fn(serialized_tf_examples):
    """Returns the output to be used in the serving signature."""
    feature_spec = tf_transform_output.raw_feature_spec()
    feature_spec.pop(_LABEL_KEY)
    parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
    transformed_features = model.tft_layer(parsed_features)
    return model(transformed_features)

  return serve_tf_examples_fn


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_transformed_name(_LABEL_KEY)),
      tf_transform_output.transformed_metadata.schema)


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying taxi data.

  Args:
    hidden_units: [int], the layer sizes of the DNN (input layer first).

  Returns:
    A keras Model.
  """
  real_valued_columns = [
      tf.feature_column.numeric_column(key)
      for key in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
  ]
  
  categorical_columns = [
      tf.feature_column.indicator_column(
          tf.feature_column.categorical_column_with_identity(
              key, num_buckets=len(boundaries)+1
          )
      ) for key, boundaries in zip(
          _transformed_names(_BUCKET_FEATURE_KEYS), _BUCKET_FEATURE_BOUNDARIES
      )
  ]
    
  categorical_columns += [
      tf.feature_column.indicator_column(
          tf.feature_column.categorical_column_with_identity( 
              key,
              num_buckets=num_buckets,
              default_value=0
          )
      ) for key, num_buckets in zip(
              _transformed_names(_VOCAB_FEATURE_KEYS),
              _MAX_CATEGORICAL_FEATURE_VALUES
      )      
  ]

  model = _dnn_regressor(
      columns=real_valued_columns+categorical_columns,
      dnn_hidden_units=[128, 64])
  return model


def _dnn_regressor(columns, dnn_hidden_units):
  """Build a simple keras wide and deep model.

  Args:
    wide_columns: Feature columns wrapped in indicator_column for wide (linear)
      part of the model.
    deep_columns: Feature columns for deep part of the model.
    dnn_hidden_units: [int], the layer sizes of the hidden DNN.

  Returns:
    A Wide and Deep Keras model
  """
  # Following values are hard coded for simplicity in this example,
  # However prefarably they should be passsed in as hparams.

  # Keras needs the feature definitions at compile time.  
  input_layers = {
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype=tf.float32)
      for colname in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
  }
  input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _transformed_names(_VOCAB_FEATURE_KEYS)
  })
  input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _transformed_names(_BUCKET_FEATURE_KEYS)
  })  

  # TODO(b/161952382): Replace with Keras preprocessing layers.
  output = tf.keras.layers.DenseFeatures(columns)(input_layers)
  for numnodes in dnn_hidden_units:
    output = tf.keras.layers.Dense(numnodes, activation='tanh')(output)
  
  output = tf.keras.layers.Dense(1)(output)

  model = tf.keras.Model(input_layers, output)
  model.compile(
      loss='mean_squared_error',
      optimizer=tf.keras.optimizers.Adam(lr=0.001)
  )
  model.summary(print_fn=absl.logging.info)

  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  absl.logging.info("="*50)
  absl.logging.info("Printing the tfx.components.FnArgs object")  
  absl.logging.info(fn_args)
  absl.logging.info("="*50)

  tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

  train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor, 
                            tf_transform_output, 40)
  eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor, 
                           tf_transform_output, 40)

  model = _build_keras_model()
  
  os.makedirs(os.path.join('data','model_performance'), exist_ok=True)

  csv_callback = tf.keras.callbacks.CSVLogger(
    os.path.join('data', 'model_performance', 'model1.csv'), append=False
  )

  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      epochs=10,
      callbacks=[csv_callback]
  )

  # https://www.tensorflow.org/tfx/serving/signature_defs
  signatures = {
      'serving_default':
          _get_serve_tf_examples_fn(
              model, tf_transform_output
          ).get_concrete_function(
              tf.TensorSpec(
                  shape=[None],
                  dtype=tf.string,
                  name='examples'
              )
          ),
      tf.compat.v1.saved_model.signature_constants.REGRESS_METHOD_NAME:
          _get_serve_tf_examples_fn(
              model, tf_transform_output
          ).get_concrete_function(
              tf.TensorSpec(
                  shape=[None],
                  dtype=tf.string,
                  name='examples'
              )
          ),
  }
  model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)


Overwriting forest_fires_trainer.py


### Understanding `tf.feature_column`

In [108]:
# https://www.tensorflow.org/api_docs/python/tf/feature_column/numeric_column

a = tf.feature_column.numeric_column("a")
x = tf.keras.layers.DenseFeatures(a)({'a': [0.5, 0.6]})
print(x)

b = tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_identity('b', num_buckets=10))
y = tf.keras.layers.DenseFeatures(b)({'b': [5, 2]})
print(y)



tf.Tensor(
[[0.5]
 [0.6]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]], shape=(2, 10), dtype=float32)


### Running the trainer

In [109]:
from tfx.components import Trainer
from tfx.proto import trainer_pb2
import tensorflow.keras.backend as K

K.clear_session()


trainer = Trainer(
    module_file=os.path.abspath("forest_fires_trainer.py"),
    transformed_examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(num_steps=100),
    eval_args=trainer_pb2.EvalArgs(num_steps=100))
context.run(trainer)


INFO:absl:Generating ephemeral wheel package for '/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/forest_fires_trainer.py' (including modules: ['forest_fires_constants', 'forest_fires_transform', 'forest_fires_trainer']).
INFO:absl:User module package has hash fingerprint version 626064dad212184db8b5fe12b96d6ebcf4204f9254387758f58ee8edf2b06004.
INFO:absl:Executing: ['/home/thushv89/anaconda3/envs/manning.tf2.tfx/bin/python', '/tmp/tmp_1lmtf1o/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp15dpbycg', '--dist-dir', '/tmp/tmp7r69of74']
INFO:absl:Successfully built user code wheel distribution at '/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/_wheels/tfx_user_code_Trainer-0.0+626064dad212184db8b5fe12b96d6ebcf4204f9254387758f58ee8edf2b06004-py3-none-any.whl'; target user module is 'forest_fires_trainer'.
INFO:absl:Full user module path is 'forest_fires_trainer@/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/exampl

INFO:absl:Feature wind_xf has a shape . Setting to DenseTensor.
INFO:absl:Model: "model"
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Layer (type)                    Output Shape         Param #     Connected to                     
INFO:absl:==================================================================================================
INFO:absl:DC_xf (InputLayer)              [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:DMC_xf (InputLayer)             [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:FFMC_xf (InputLayer)            [(None,)]            0                                            
INFO:absl:_____________________________

Epoch 1/10
100/100 [==============================] - ETA: 39s - loss: 43840.07 - ETA: 0s - loss: 7937.2805 - ETA: 0s - loss: 6942.85 - 1s 5ms/step - loss: 6650.6988 - val_loss: 540.5629
Epoch 2/10
100/100 [==============================] - ETA: 0s - loss: 1039.97 - ETA: 0s - loss: 5841.33 - ETA: 0s - loss: 5816.50 - 0s 3ms/step - loss: 5843.9747 - val_loss: 550.3081
Epoch 3/10
100/100 [==============================] - ETA: 0s - loss: 835.907 - ETA: 0s - loss: 5995.13 - ETA: 0s - loss: 5884.64 - 0s 3ms/step - loss: 5837.2405 - val_loss: 557.0079
Epoch 4/10
100/100 [==============================] - ETA: 0s - loss: 994.313 - ETA: 0s - loss: 5718.51 - ETA: 0s - loss: 5778.46 - ETA: 0s - loss: 5774.92 - 0s 3ms/step - loss: 5773.1266 - val_loss: 549.7269
Epoch 5/10
100/100 [==============================] - ETA: 0s - loss: 696.515 - ETA: 0s - loss: 4933.01 - ETA: 0s - loss: 5246.45 - 0s 3ms/step - loss: 5373.1893 - val_loss: 542.7257
Epoch 6/10
100/100 [==============================] - E

INFO:tensorflow:Assets written to: /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/5/Format-Serving/assets


INFO:tensorflow:Assets written to: /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/5/Format-Serving/assets
INFO:absl:Training complete. Model written to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/5/Format-Serving. ModelRun written to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model_run/5
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 5
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 7
        type_id: 14
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/5"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 8
        type_id: 15
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model_run/5"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 15
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## Evaluate the trained model

### Load the model

In [110]:
model = load_model(os.path.join(trainer.outputs["model"]._artifacts[0].uri, "Format-Serving"))
model.summary()

Exception ignored in: <bound method CapturableResourceDeleter.__del__ of <tensorflow.python.training.tracking.tracking.CapturableResourceDeleter object at 0x7f3c0c0b9240>>
Traceback (most recent call last):
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/training/tracking/tracking.py", line 208, in __del__
    self._destroy_resource()
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 726, in _initialize
    *args, **kwds))
  File "/home/thushv89/anaconda3/envs/manning.t

tensorflow.python.framework.errors_impl.InvalidArgumentError: 'func' argument to TF_GraphCopyFunction cannot be null
Exception ignored in: <bound method CapturableResourceDeleter.__del__ of <tensorflow.python.training.tracking.tracking.CapturableResourceDeleter object at 0x7f3b9f5ea278>>
Traceback (most recent call last):
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/training/tracking/tracking.py", line 208, in __del__
    self._destroy_resource()
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python


Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f3bcbc509b0> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f3b9f7220b8>).



Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f3bcbc509b0> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f3b9f7220b8>).


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
DC_xf (InputLayer)              [(None,)]            0                                            
__________________________________________________________________________________________________
DMC_xf (InputLayer)             [(None,)]            0                                            
__________________________________________________________________________________________________
FFMC_xf (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
ISI_xf (InputLayer)             [(None,)]            0                                            
______________________________________________________________________________________________

### Generate samples using the test file input

In [111]:
#https://www.tensorflow.org/tfx/transform/api_docs/python/tft/TFTransformOutput
    
import tensorflow_transform as tft
tf_transform_output = tft.TFTransformOutput(transform.outputs["transform_graph"]._artifacts[0].uri)

print("="*10, " Feature Specifications ", "="*10)
print(tf_transform_output.raw_feature_spec()) 
print("="*50, "\n")

# Load data
test_df = pd.read_csv(os.path.join(test_path, 'forestfires.csv'), index_col=None, header=0)

# Sample contains a dictionary of feature_name -> ragged tensor
sample = {}
for col, val in test_df.to_dict(orient='list').items():
    
    if col=='area':
        continue
    
    # Transformation came from https://github.com/tensorflow/transform/blob/d401253873a2079c245049572e8488347adb3cd4/examples/census_example_v2.py#L92
    if isinstance(val[0], int):
        sample[col] = tf.RaggedTensor.from_tensor(tf.cast(tf.expand_dims(val, -1), dtype='int64')).to_sparse()
        
    else:
        sample[col] = tf.RaggedTensor.from_tensor(tf.expand_dims(val, -1)).to_sparse()

print("="*10, " Sample ", "="*10)
print(sample)
print("="*50, "\n")



==========  Feature Specifications  ==========
{'day': VarLenFeature(dtype=tf.string), 'month': VarLenFeature(dtype=tf.string), 'DC': VarLenFeature(dtype=tf.float32), 'DMC': VarLenFeature(dtype=tf.float32), 'FFMC': VarLenFeature(dtype=tf.float32), 'ISI': VarLenFeature(dtype=tf.float32), 'RH': VarLenFeature(dtype=tf.int64), 'X': VarLenFeature(dtype=tf.int64), 'Y': VarLenFeature(dtype=tf.int64), 'area': VarLenFeature(dtype=tf.float32), 'rain': VarLenFeature(dtype=tf.float32), 'temp': VarLenFeature(dtype=tf.float32), 'wind': VarLenFeature(dtype=tf.float32)}

==========  Sample  ==========
{'X': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f3b9f64ab38>, 'Y': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f3c0ca2bc50>, 'month': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f3c0ca2b5f8>, 'day': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f3c0ca2b208>, 'FFMC': <tensorflow.python.framework.sparse_tens

### Predict with the loaded model

In [112]:
transformed_features = tf_transform_output.transform_features_layer()(sample)

predictions = model.predict(transformed_features)

print(predictions)

[[36.917007 ]
 [ 8.445333 ]
 [16.752258 ]
 [10.180081 ]
 [11.737633 ]
 [ 9.407103 ]
 [ 8.644513 ]
 [ 5.071876 ]
 [17.004484 ]
 [-0.4543419]
 [25.161861 ]
 [-2.499673 ]
 [ 6.3441176]
 [ 2.1701598]
 [ 9.434877 ]
 [ 8.307977 ]
 [ 8.067101 ]
 [ 7.246016 ]
 [ 2.3090577]
 [-1.5537155]
 [12.347721 ]
 [15.146367 ]
 [ 0.8304266]
 [12.201796 ]
 [12.587576 ]
 [ 9.572004 ]]


## Adapting the schema and treating anomalies in the data

In [113]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'day',STRING,required,single,'day'
'month',STRING,required,single,'month'
'DC',FLOAT,required,single,-
'DMC',FLOAT,required,single,-
'FFMC',FLOAT,required,single,-
'ISI',FLOAT,required,single,-
'RH',INT,required,single,-
'X',INT,required,single,-
'Y',INT,required,single,-


,Values
Domain,
'day',"'fri', 'mon', 'sat', 'sun', 'thu', 'tue', 'wed'"
'month',"'apr', 'aug', 'dec', 'feb', 'jan', 'jul', 'jun', 'mar', 'may', 'oct', 'sep', 'nov'"


### Adapting the schema to identify anomalies

These various attributes are defined at `https://github.com/tensorflow/metadata/blob/master/tensorflow_metadata/proto/v0/schema.proto`

In [114]:
# Use this for detecting what anomly conditions are detected
#https://www.tensorflow.org/tfx/data_validation/anomalies
    
schema = tfdv.load_schema_text(os.path.join(schema_gen.outputs['schema']._artifacts[0].uri, 'schema.pbtxt'))
statistics = tfdv.generate_statistics_from_csv(os.path.join('data', 'csv', 'train', 'forestfires.csv'))

y_feature = tfdv.get_feature(schema, 'Y')
isi_feature = tfdv.get_feature(schema, 'ISI')
ffmc_feature = tfdv.get_feature(schema, 'FFMC')
rain_feature = tfdv.get_feature(schema, 'rain')
area_feature = tfdv.get_feature(schema, 'area')

y_feature.int_domain.max = 7
isi_feature.float_domain.max = 30.0
ffmc_feature.float_domain.min = 60.0
rain_feature.float_domain.max = 1.0
area_feature.float_domain.max = 75.0


anomalies = tfdv.validate_statistics(statistics = statistics, schema = schema)
#anomalies = tfdv.validate_examples_in_csv(os.path.join('data', 'csv', 'forestfires.csv'))

tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'rain',Out-of-range values,Unexpectedly high value: 6.4>1(upto six significant digits)
'Y',Out-of-range values,Unexpectedly large value: 9.
'ISI',Out-of-range values,Unexpectedly high value: 56.1>30(upto six significant digits)
'FFMC',Out-of-range values,Unexpectedly low values: 18.7<60(upto six significant digits)
'area',Out-of-range values,Unexpectedly high value: 1090.84>75(upto six significant digits)


## Overwrite the schema

In [115]:
tfdv.write_schema_text(schema, os.path.join(schema_gen.outputs['schema']._artifacts[0].uri, 'schema.pbtxt'))

## Checking examples

In [116]:
from tfx.components import ExampleValidator


example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])

context.run(example_validator)


INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Nonempty beam arg extra_packages already includes dependency
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/ExampleValidator/anomalies/6/Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/ExampleValidator/anomalies/6/Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 6
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 9
        type_id: 17
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/ExampleValidator/anomalies/6"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 17
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### View anomalies

In [117]:
context.show(example_validator.outputs['anomalies'])

,Anomaly short description,Anomaly long description
Feature name,,
'ISI',Out-of-range values,Unexpectedly high value: 56.1>30(upto six significant digits)
'area',Out-of-range values,Unexpectedly high value: 1090.84>75(upto six significant digits)
'FFMC',Out-of-range values,Unexpectedly low values: 18.7<60(upto six significant digits)
'rain',Out-of-range values,Unexpectedly high value: 6.4>1(upto six significant digits)
'Y',Out-of-range values,Unexpectedly large value: 9.


,Anomaly short description,Anomaly long description
Feature name,,
'Y',Out-of-range values,Unexpectedly large value: 9.
'area',Out-of-range values,Unexpectedly high value: 212.88>75(upto six significant digits)


### Remove anomalies

In [118]:
df = pd.read_csv(os.path.join('data','csv', 'train','forestfires.csv'), header=0)

print("Original shape: {}".format(df.shape))
valid_df = df[(df["FFMC"]>=60)&(df["ISI"]<=30)&(df["Y"]<=7)&(df["area"]<=75)&(df["rain"]<=1)]
print("After anomalies removed shape: {}".format(valid_df.shape))
os.makedirs(os.path.join('data','csv_valid', 'train'),exist_ok=True)
valid_df.to_csv(os.path.join('data','csv_valid','train','forestfires.csv'), header=True, index=False)

Original shape: (491, 13)
After anomalies removed shape: (465, 13)


### View statistics of data side by side (before and after removing anomalies)

In [119]:

dataset1_stats = tfdv.generate_statistics_from_csv(
  data_location=os.path.join('data', 'csv', 'train', 'forestfires.csv'), delimiter=',')
dataset2_stats = tfdv.generate_statistics_from_csv(
  data_location=os.path.join('data', 'csv_valid', 'train', 'forestfires.csv'), delimiter=',')

tfdv.visualize_statistics(
  lhs_statistics=dataset1_stats, lhs_name='Original data',
  rhs_statistics=dataset2_stats, rhs_name='Data without anomalies'
)


## Transform data (after removing anomalies)

In [120]:
from tfx.components import Transform

# Define the example generator
example_gen = CsvExampleGen(input_base=os.path.join('data', 'csv_valid', 'train'))
# Run the example generator
context.run(example_gen)

# Define the statistic generator
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
# Run the statistic  generator
context.run(statistics_gen)

example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])

context.run(example_validator)

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath('forest_fires_transform.py'))

context.run(transform)



INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Nonempty beam arg extra_packages already includes dependency
INFO:absl:Generating examples.
INFO:absl:Processing input csv data data/csv_valid/train/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Nonempty beam arg extra_packages already includes dependency
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_p

INFO:absl:Feature day has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature month has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature FFMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature ISI has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature RH has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature X has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Y has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature area has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature rain has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature temp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature wind has no shape. Setting to VarLenSparseTensor.


INFO:absl:Feature day has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature month has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature FFMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature ISI has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature RH has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature X has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Y has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature area has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature rain has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature temp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature wind has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature day has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature month has no shape. Setting to VarLenSparse

Exception ignored in: <bound method CapturableResourceDeleter.__del__ of <tensorflow.python.training.tracking.tracking.CapturableResourceDeleter object at 0x7f3b9e0e3828>>
Traceback (most recent call last):
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/training/tracking/tracking.py", line 208, in __del__
    self._destroy_resource()
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 726, in _initialize
    *args, **kwds))
  File "/home/thushv89/anaconda3/envs/manning.t

    result = self._call(*args, **kwds)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 726, in _initialize
    *args, **kwds))
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/function.py", line 2969, in _get_concrete_function_internal_garbage_collected
    graph_function, _ = self._maybe_define_function(args, kwargs)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/function.py", line 3361, in _maybe_define_function
    graph_function = self._create_graph_function(args, kwargs)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/functi

...

INFO:tensorflow:Assets written to: /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Transform/transform_graph/10/.temp_path/tftransform_tmp/fcdab6af04af45f59b2f3cb203de8739/assets


INFO:tensorflow:Assets written to: /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Transform/transform_graph/10/.temp_path/tftransform_tmp/739609a97e394e3e85ca466ca1616092/assets


INFO:tensorflow:Assets written to: /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Transform/transform_graph/10/.temp_path/tftransform_tmp/739609a97e394e3e85ca466ca1616092/assets


INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 10
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 13
        type_id: 11
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Transform/transform_graph/10"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 14
        type_id: 5
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Transform/transformed_examples/10"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 15
        type_id: 12
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Transform/updated_analyzer_cache/10"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 12
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [121]:
context.show(statistics_gen.outputs['statistics'])

In [122]:
context.show(example_validator.outputs['anomalies'])

## Train the model (after removing anomalies)

In [123]:
from tfx.components import Trainer
from tfx.proto import trainer_pb2

trainer = Trainer(
    module_file=os.path.abspath("forest_fires_trainer.py"),
    transformed_examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(num_steps=100),
    eval_args=trainer_pb2.EvalArgs(num_steps=100))
context.run(trainer)


INFO:absl:Generating ephemeral wheel package for '/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/forest_fires_trainer.py' (including modules: ['forest_fires_constants', 'forest_fires_transform', 'forest_fires_trainer']).
INFO:absl:User module package has hash fingerprint version 626064dad212184db8b5fe12b96d6ebcf4204f9254387758f58ee8edf2b06004.
INFO:absl:Executing: ['/home/thushv89/anaconda3/envs/manning.tf2.tfx/bin/python', '/tmp/tmpekchbjwo/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp7a320e9t', '--dist-dir', '/tmp/tmp710oosal']
INFO:absl:Successfully built user code wheel distribution at '/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/_wheels/tfx_user_code_Trainer-0.0+626064dad212184db8b5fe12b96d6ebcf4204f9254387758f58ee8edf2b06004-py3-none-any.whl'; target user module is 'forest_fires_trainer'.
INFO:absl:Full user module path is 'forest_fires_trainer@/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/exampl

INFO:absl:Feature wind_xf has a shape . Setting to DenseTensor.
INFO:absl:Model: "model_1"
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Layer (type)                    Output Shape         Param #     Connected to                     
INFO:absl:==================================================================================================
INFO:absl:DC_xf (InputLayer)              [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:DMC_xf (InputLayer)             [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:FFMC_xf (InputLayer)            [(None,)]            0                                            
INFO:absl:___________________________

Epoch 1/10
100/100 [==============================] - ETA: 38s - loss: 154.64 - ETA: 0s - loss: 161.1458 - ETA: 0s - loss: 155.151 - 1s 5ms/step - loss: 150.7824 - val_loss: 140.3804
Epoch 2/10
100/100 [==============================] - ETA: 0s - loss: 128.129 - ETA: 0s - loss: 121.602 - ETA: 0s - loss: 124.840 - 0s 3ms/step - loss: 125.6834 - val_loss: 141.4916
Epoch 3/10
100/100 [==============================] - ETA: 0s - loss: 123.653 - ETA: 0s - loss: 131.420 - ETA: 0s - loss: 128.675 - 0s 3ms/step - loss: 127.1231 - val_loss: 142.3460
Epoch 4/10
100/100 [==============================] - ETA: 0s - loss: 72.36 - ETA: 0s - loss: 117.764 - ETA: 0s - loss: 117.469 - 0s 3ms/step - loss: 116.9093 - val_loss: 145.6050
Epoch 5/10
100/100 [==============================] - ETA: 0s - loss: 142.638 - ETA: 0s - loss: 104.042 - ETA: 0s - loss: 104.320 - 0s 3ms/step - loss: 104.0363 - val_loss: 147.7969
Epoch 6/10
100/100 [==============================] - ETA: 0s - loss: 35.05 - ETA: 0s - los

Exception ignored in: <bound method CapturableResourceDeleter.__del__ of <tensorflow.python.training.tracking.tracking.CapturableResourceDeleter object at 0x7f3b96fb4198>>
Traceback (most recent call last):
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/training/tracking/tracking.py", line 208, in __del__
    self._destroy_resource()
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 726, in _initialize
    *args, **kwds))
  File "/home/thushv89/anaconda3/envs/manning.t

tensorflow.python.framework.errors_impl.InvalidArgumentError: 'func' argument to TF_GraphCopyFunction cannot be null
Exception ignored in: <bound method CapturableResourceDeleter.__del__ of <tensorflow.python.training.tracking.tracking.CapturableResourceDeleter object at 0x7f3b96fb4048>>
Traceback (most recent call last):
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/training/tracking/tracking.py", line 208, in __del__
    self._destroy_resource()
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python

    result = self._call(*args, **kwds)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 726, in _initialize
    *args, **kwds))
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/function.py", line 2969, in _get_concrete_function_internal_garbage_collected
    graph_function, _ = self._maybe_define_function(args, kwargs)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/function.py", line 3361, in _maybe_define_function
    graph_function = self._create_graph_function(args, kwargs)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/functi

    graph_function = self._create_graph_function(args, kwargs)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/function.py", line 3206, in _create_graph_function
    capture_by_value=self._capture_by_value),
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/framework/func_graph.py", line 990, in func_graph_from_py_func
    func_outputs = python_func(*func_args, **func_kwargs)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 634, in wrapped_fn
    out = weak_wrapped_fn().__wrapped__(*args, **kwds)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/saved_model/function_deserialization.py", line 253, in restored_function_body
    return _call_concrete_function(function, inputs)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packag

INFO:tensorflow:Assets written to: /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/11/Format-Serving/assets


INFO:tensorflow:Assets written to: /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/11/Format-Serving/assets
INFO:absl:Training complete. Model written to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/11/Format-Serving. ModelRun written to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model_run/11
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 11
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 16
        type_id: 14
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/11"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 17
        type_id: 15
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model_run/11"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 15
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## Printing the location of the output file

In [124]:
trainer.outputs['model']._artifacts[0].uri

'/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/11'

## Step 6: Validating the Infrastructure of the model

In [54]:
from tfx.components import InfraValidator
from tfx.proto import infra_validator_pb2
infra_validator = InfraValidator(
    model=trainer.outputs['model'],
    # This is the source for the data that will be used to build a request.
    examples=example_gen.outputs['examples'],
    serving_spec=infra_validator_pb2.ServingSpec(
        # Depending on what kind of model server you're using, RequestSpec
        # should specify the compatible one.
        tensorflow_serving=infra_validator_pb2.TensorFlowServing(tags=['2.4.1-gpu']),
        local_docker=infra_validator_pb2.LocalDockerConfig(
            client_base_url="unix:///var/run/docker.sock", client_api_version="1.40"
        ),
    ),
    request_spec=infra_validator_pb2.RequestSpec(
        # InfraValidator will look at how "classification" signature is defined
        # in the model, and automatically convert some samples from `examples`
        # artifact to prediction RPC requests.
        tensorflow_serving=infra_validator_pb2.TensorFlowServingRequestSpec(
            signature_names=['serving_default']
        ),
        num_examples=5  # How many requests to make.
    )
)

context.run(infra_validator)

INFO:absl:Running driver for InfraValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for InfraValidator
INFO:absl:Nonempty beam arg extra_packages already includes dependency
INFO:absl:InfraValidator will be run in LOAD_AND_QUERY mode.
INFO:absl:tag_set is not given. Using {'serve'} instead.
INFO:absl:Creating temp directory at /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/.temp/9/
INFO:absl:Starting infra validation (attempt 1/5).
INFO:absl:Starting LocalDockerRunner(image: tensorflow/serving:2.4.1-gpu).
INFO:absl:Running container with parameter {'auto_remove': True, 'detach': True, 'publish_all_ports': True, 'image': 'tensorflow/serving:2.4.1-gpu', 'environment': {'MODEL_NAME': 'infra-validation-model', 'MODEL_BASE_PATH': '/model'}, 'mounts': [{'Target': '/model/infra-validation-model/1', 'Source': '/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/.temp/9/i

ExecutionResult(
    component_id: InfraValidator
    execution_id: 9
    outputs:
        blessing: Channel(
            type_name: InfraBlessing
            artifacts: [Artifact(artifact: id: 10
        type_id: 19
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/InfraValidator/blessing/9"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "InfraValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 19
        name: "InfraBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## Step 7: Validating the model

In [37]:
import tensorflow_model_analysis as tfma

eval_config = tfma.EvalConfig(
    model_specs=[
        # This assumes a serving model with signature 'serving_default'. If
        # using estimator based EvalSavedModel, add signature_name: 'eval' and 
        # remove the label_key.
        tfma.ModelSpec(label_key='area')
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            # To add validation thresholds for metrics saved with the model,
            # add them keyed by metric name to the thresholds map.
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='MeanSquaredError',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          upper_bound={'value': 7500.0}),
                      # Change threshold will be ignored if there is no
                      # baseline model resolved from MLMD (first run).
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.LOWER_IS_BETTER,
                          absolute={'value': 1e-10})))
            ]
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec(),
        # Data can be sliced along a feature column. In this case, data is
        # sliced along feature column month.
        tfma.SlicingSpec(feature_keys=['month'])
    ])


In [38]:
from tfx import v1 as tfx

model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')

context.run(model_resolver)



INFO:absl:Running driver for latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 10
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [39]:
from tfx.components import Evaluator

evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
context.run(evaluator)


INFO:absl:Running driver for Evaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Evaluator
INFO:absl:Nonempty beam arg extra_packages already includes dependency
ERROR:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "ExampleCount"\n        },\n        {\n          "class_name": "MeanSquaredError",\n          "threshold": {\n            "change_threshold": {\n              "absolute": 1e-10,\n              "direction": "LOWER_IS_BETTER"\n            },\n            "value_threshold": {\n              "upper_bound": 7500.0\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "area"\n    }\n  ],\n  "slicing_specs": [\n    {},\n    {\n      "feature_keys": [\n        "month"\n      ]\n    }\n  ]\n}', 'feature_slicing_spec': None, 'fairness_indicator_thresholds': None, 'example_splits': 'null', 'module_file': None, '

Exception ignored in: <bound method CapturableResourceDeleter.__del__ of <tensorflow.python.training.tracking.tracking.CapturableResourceDeleter object at 0x7f548dd9f0f0>>
Traceback (most recent call last):
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/training/tracking/tracking.py", line 208, in __del__
    self._destroy_resource()
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 726, in _initialize
    *args, **kwds))
  File "/home/thushv89/anaconda3/envs/manning.t


Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f5448c721d0> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f5448d48668>).



Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f5448c721d0> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f5448d48668>).


INFO:absl:The 'example_splits' parameter is not set, using 'eval' split.
INFO:absl:Evaluating model.
ERROR:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "ExampleCount"\n        },\n        {\n          "class_name": "MeanSquaredError",\n          "threshold": {\n            "change_threshold": {\n              "absolute": 1e-10,\n              "direction": "LOWER_IS_BETTER"\n            },\n            "value_threshold": {\n              "upper_bound": 7500.0\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "area"\n    }\n  ],\n  "slicing_specs": [\n    {},\n    {\n      "feature_keys": [\n        "month"\n      ]\n    }\n  ]\n}', 'feature_slicing_spec': None, 'fairness_indicator_thresholds': None, 'example_splits': 'null', 'module_file': None, 'module_path': None} 'custom_extractors'
INFO:absl:Request was made to ignore the baseline ModelSpec and a

...


Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f53150c4518> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f53151ace48>).



Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f531494a748> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f53149a8748>).



Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f531494a748> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f53149a8748>).


INFO:absl:Evaluation complete. Results written to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Evaluator/evaluation/11.
INFO:absl:Checking validation results.
INFO:absl:Blessing result True written to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Evaluator/blessing/11.
INFO:absl:Running publisher for Evaluator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Evaluator
    execution_id: 11
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 16
        type_id: 22
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Evaluator/evaluation/11"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 17
        type_id: 23
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Evaluator/blessing/11"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/8"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 13
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [40]:
print(evaluator.outputs.blessing.get()[0].uri)

/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Evaluator/blessing/11


## Viewing the validation results

In [41]:
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))


validation_ok: true
validation_details {
  slicing_details {
    slicing_spec {
    }
    num_matching_slices: 12
  }
}



## Step 8: Pushing the model to the deployment environment

In [42]:
from tfx.components import Pusher
from tfx.proto import pusher_pb2

pusher = Pusher(
  model=trainer.outputs['model'],
  model_blessing=evaluator.outputs['blessing'],
  infra_blessing=infra_validator.outputs['blessing'],
  push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory=os.path.join('forestfires-model-pushed'))
  )
)
context.run(pusher)

INFO:absl:Running driver for Pusher
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Pusher
INFO:absl:Nonempty beam arg extra_packages already includes dependency
INFO:absl:Model version: 1625380180
INFO:absl:Model written to serving path forestfires-model-pushed/1625380180.
INFO:absl:Model pushed to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Pusher/pushed_model/12.
INFO:absl:Running publisher for Pusher
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Pusher
    execution_id: 12
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 18
        type_id: 25
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Pusher/pushed_model/12"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "forestfires-model-pushed/1625380180"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1625380180"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 25
        name: "PushedModel"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## List metadata of the API

In [191]:
json_response = requests.get(
    'http://localhost:8501/v1/models/forest_fires_model/metadata'
)
predictions = json.loads(json_response.text)

print(predictions)

{'model_spec': {'name': 'forest_fires_model', 'signature_name': '', 'version': '1'}, 'metadata': {'signature_def': {'signature_def': {'serving_default': {'inputs': {'examples': {'dtype': 'DT_STRING', 'tensor_shape': {'dim': [{'size': '-1', 'name': ''}], 'unknown_rank': False}, 'name': 'serving_default_examples:0'}}, 'outputs': {'output_0': {'dtype': 'DT_FLOAT', 'tensor_shape': {'dim': [{'size': '-1', 'name': ''}, {'size': '1', 'name': ''}], 'unknown_rank': False}, 'name': 'StatefulPartitionedCall_10:0'}}, 'method_name': 'tensorflow/serving/predict'}, '__saved_model_init_op': {'inputs': {}, 'outputs': {'__saved_model_init_op': {'dtype': 'DT_INVALID', 'tensor_shape': {'dim': [], 'unknown_rank': True}, 'name': 'NoOp'}}, 'method_name': ''}}}}}


## Predict with the API

In [198]:
# curl http://localhost:8501/v1/models/forest_fires_model
# curl -H "Content-Type: application/json" --data @data/request/request_body_regress.json http://localhost:8501/v1/models/forest_fires_mode:regress

import base64

req_body = {
  "signature_name": "serving_default",

  "instances": 
    [
            str(base64.b64encode(
                b"{\"X\": 7,\"Y\": 4,\"month\":\"oct\",\"day\":\"fri\",\"FFMC\":60,\"DMC\":30,\"DC\":200,\"ISI\":9,\"temp\":30,\"RH\":50,\"wind\":10,\"rain\":0}]")
               )
    ]
    
}


import json
data = json.dumps(req_body)

import requests
headers = {"content-type": "application/json"}
json_response = requests.post(
    'http://localhost:8501/v1/models/forest_fires_model:predict', data=data, headers=headers
)
predictions = json.loads(json_response.text)

print(predictions)


{'predictions': [[2.77522683]]}


## Define TFX steps as a single pipeline